In [150]:
!pip install beautifulsoup4 requests urllib3 selenium pandas elasticsearch

  Using cached https://files.pythonhosted.org/packages/bd/3b/a2f4a4f1f7578ceaff490961753a75984efc17c17b1f6f59c3a866debeca/elastic_transport-8.4.0-py3-none-any.whl


ERROR: elastic-transport 8.4.0 has requirement urllib3<2,>=1.26.2, but you'll have urllib3 2.0.1 which is incompatible.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# yt extract

In [ ]:
# TODO
  # 1. Add following additional fileds in the video dictionary : 'playlist_url', 'video_playlist_title', 'video_order_number'
  # 2. Save the list of dictionary in ES where each record will be an independent video containing all video fileds:
  #  {
  #       '_id',   
  #       'video_url', 
  #       'video_title',
  #       'video_order_number',  
  #       'iframe_embedding_text', 
  #       'video_author_name', 
  #       'date_of_publish',
  #       'video_view_count',

  #       'playlist_url', 
  #       'video_playlist_title', 
  #       'video_review':[{"comment":"", "review_date":"", "review_rating":""}]  #this is different than youtube user comments
  #  } 
  # 3. Search query to get all videos order by 'video_order_number' for a given playlist url with all details.


Check __selenium_config.json__ and change the param for os to linux during colab test

## main code

### extraction

In [156]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as firefox_opts
from selenium.webdriver.chrome.options import Options as chrome_opts
import time
import time
import json  
import re
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

"""
	Noise removal
"""
def remove_tags(soup):
	regex_header = re.compile('^header|^navbar')
	regex_footer = re.compile('footer|mz|lightbox|nav|foot|shariff|bottom|enquiry|related-content|LicenseInfo')
	regex_social_media = re.compile('social|subscribe|affiliateCard|tag|notification|announcement|author|infobox|comment')
	regex_button = re.compile('button|btn')
	regex_em = re.compile('widget')
	regex_css = re.compile('css')
	regex_cookie = re.compile('cookie|cc-banner')
	regex_sidebar = re.compile('sidebar')
	regex_modal = re.compile('modal|popup')
	regex_references = re.compile('References|article-info-section')
	# soup.find_all("div", {"class" : regex})

	div_class_header = soup.findAll("div", {"class" : regex_header})
	div_class_footer = soup.findAll(["div","section"],{"class" : regex_footer})
	div_id_sidebars = soup.findAll(["div","aside"], {"id" : regex_sidebar})
	div_class_sidebars = soup.findAll(["div","aside"], {"class":regex_sidebar})
	dragdiv = soup.findAll("div", {"id" :  "dragdiv"})
	divcom  = soup.findAll("div",{'class' : 'comments'})
	div_class_em = soup.findAll("div",{"class" : regex_em})
	modals = soup.findAll(["div","section"],{"class":regex_modal})
	cookie = soup.findAll("div",{"data-nosnippet":"true"})
	div_id_social = soup.findAll("div", {"id" : regex_social_media})
	div_class_social = soup.findAll("div", {"class":regex_social_media})
	a_button_cls = soup.findAll('a',{"class":regex_button})
	a_button_id = soup.findAll('a',{"id":regex_button})
	div_sec_reference = soup.findAll(['div','section'],{"data-title":regex_references})
	div_sec_reference_id = soup.findAll(['div','section'],{"id":regex_references})
	scripts = soup.findAll(['head','nav','script', 'footer','aside','img','figure','svg','button','app-related-posts'])

	# Save links & style - append it at the end of <body> tag 
	css_obj = soup.find_all('style')
	css_reg_obj = soup.find_all('link', {'href' : regex_css})
	for item in css_obj:
		soup.html.body.append(item)
	for link in css_reg_obj:
		soup.html.body.append(link)

	# Remove unneccessary tags
	for divh in div_class_header:
		divh.decompose()
	for divr in div_sec_reference:
		divr.decompose()
	for divh in div_sec_reference_id:
		divh.decompose()
	for divf in div_class_footer:
		divf.decompose()
	for divside in div_id_sidebars:
		# print(divside)
		divside.decompose()
	for divdrag in dragdiv:
		print(divdrag)
		divdrag.decompose()
	for com in divcom:
		com.decompose()    
	for match in cookie:
		match.decompose()
	for match in modals:
		match.decompose()
	for match in div_class_sidebars:
		match.decompose()
	for match in div_id_social:
		match.decompose()
	for match in div_class_social:
		match.decompose()
	for match in a_button_cls:
		match.decompose()
	for match in a_button_id:
		match.decompose()
	for match in scripts:
		match.decompose()

	# return data by retrieving the tag content
	# return ' '.join(soup.stripped_strings)
	return soup



class YoutubePlExtracter:
    def __init__(self) -> None:
        #Disabling the browser window to open
        selenium_config = json.load(open("./config/selenium_config.json"))
        head_less = bool(selenium_config["headless"])
        
        firefox_option = firefox_opts()
        firefox_option.headless = head_less

        if head_less:
            chrome_option = chrome_opts()
            chrome_option.add_argument("--headless") 
        if selenium_config["os"] == "windows":
            if selenium_config["default_browser"] == "Firefox":                
                if selenium_config["system_arch"] == "64":
                    self.driver = webdriver.Firefox(options=firefox_option,executable_path=selenium_config["windows_path"]["firefox_64_bit"])
                else:
                    self.driver = webdriver.Firefox(options=firefox_option,executable_path=selenium_config["windows_path"]["firefox_32_bit"])
            elif selenium_config["default_browser"] == "Chrome":
                self.driver = webdriver.Chrome(chrome_options=chrome_option,executable_path=selenium_config["windows_path"]["chrome_32_bit"])
        if selenium_config["os"] == "linux":
            if selenium_config["default_browser"] == "Firefox":
                if selenium_config["system_arch"] == "64":
                    self.driver = webdriver.Firefox(options=firefox_option,executable_path=selenium_config["linux_path"]["firefox_64_bit"])
                else:
                    self.driver = webdriver.Firefox(options=firefox_option,executable_path=selenium_config["linux_path"]["firefox_32_bit"])
            elif selenium_config["default_browser"] == "Chrome":
                self.driver = webdriver.Chrome(chrome_options=chrome_option,executable_path=selenium_config["linux_path"]["chrome_64_bit"])
    
    
    def scroll_page(self):
        #SCRIPTINO TO SCROLL PAGE UNTIL IT ENDS
        WAIT_IN_SECONDS = 1
        last_height = self.driver.execute_script("return document.documentElement.scrollHeight")

        while True:
            # Scroll to the bottom of page
            self.driver.execute_script("window.scrollTo(0, arguments[0]);", last_height)
            # Wait for new videos to show up
            time.sleep(WAIT_IN_SECONDS)
            
            # Calculate new document height and compare it with last height
            new_height = self.driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
    
    def getlinks(self, url):        
        result = []

        self.driver.get(url)
        time.sleep(2)
        self.scroll_page()
        
        soup = bs(self.driver.page_source, features='html.parser')
        clean_soup = remove_tags(soup=soup)
        
        playlist_title = clean_soup.find('yt-formatted-string',{'id':'text'}).text

        for x in clean_soup.find_all('ytd-playlist-video-renderer',{'class':'ytd-playlist-video-list-renderer'}):
            temp = {}

            index = x.find('div',{'id':'index-container'})
            order_id = index.getText().strip("\n")
            temp['video_order_number'] = order_id

            link = x.find('a',{'id':'video-title'})
            try:
                href = link.get('href')
                if href.startswith('/watch?'):
                    # print(href)
                    temp['video_url'] = "https://www.youtube.com" + href
            except:
                temp['video_url'] = ""
            
            try:
                title = link.get('title')
                temp['video_title'] = title
            except:
                temp['video_title'] = ""
            
            channel_obj = x.find('ytd-channel-name').find('a')
            channel_name = channel_obj.getText()
            # print(channel_name)
            temp['video_author_name'] = channel_name

            sep_obj = x.find('yt-formatted-string',{'id': 'video-info'}).find_all('span')
            # print(sep_obj[0].getText())

            temp['video_view_count'] = sep_obj[0].getText()
            temp['date_of_publish'] = sep_obj[2].getText()
            temp['video_playlist_title'] = playlist_title
            temp['playlist_url'] = url
            temp['iframe_embedding_text'] = ""
            temp['video_review'] = [{"comment":"", "review_date":"", "review_rating":""}]
            result.append(temp)       
        
        self.driver.close()
        return result

### __elastic search code__

In [157]:
import json
from datetime import datetime
from elasticsearch import helpers
from elasticsearch import Elasticsearch
import os

es_config = json.load(open("config/es/es_config_local.json"))
ES_INDEX_NAME = es_config['index']
ES_INSTANCE = Elasticsearch(
    hosts=es_config['host'],
    basic_auth=(es_config['basic_auth']['user'], es_config['basic_auth']['pass']),
    ca_certs=es_config['ca_certs'],
    max_retries=1
)


class ElasticSearch:
    """
    Elastic search utility class

    Write/Upload records to index in elasticsearch index.

    Es index record structure:
        {
            '_id',   
            'video_url', 
            'video_title',
            'video_order_number',  
            'iframe_embedding_text', 
            'video_author_name', 
            'date_of_publish',
            'video_view_count',

            'playlist_url', 
            'video_playlist_title', 
            'video_review':[{"comment":"", "review_date":"", "review_rating":""}]  #this is different than youtube user comments
        } 

    Attributes:
    None
    """

    def __init__(self):
        pass

    def __add_mapping(self):
        """
        Create mapping / data dictionary for index creation in Elastic search.

        This method is a one time call function. To be run when a singleton index is created to store the records.
        Default index is 'master'.
        For new index, update the value of parameter 'index' in es_config_local.json file.

        Parameters: None

        Returns: None
        """
        es_config = json.load(open("config/es/es_mapping_config.json"))
        ES_INSTANCE.indices.create(index=ES_INDEX_NAME, ignore=400, body=es_config)
        print("Mapping created successfully")


    def write_to_es(self, data):
        """
        Sequential record insert method

        Parameters: None

        Returns: None
        """

        start = datetime.now()

        self.__add_mapping()

        print("Sequential insertion operation started")
        print("Uploading data to %s index in elastic search db...", ES_INDEX_NAME)

        count = 0
        for row in data:
            index_row_id = f"YPL__{count}" + '__' + str(int(datetime.now().microsecond))
            ES_INSTANCE.index(index=ES_INDEX_NAME, id=index_row_id, document=row)
            count += 1

        print("Records inserted:  ", count)

        end = datetime.now()
        print("Sequential insertion operation completed.... Time taken: ", (end - start))

    def get_all_links(self):
        links_qs = \
            ES_INSTANCE.search(index=ES_INDEX_NAME, body={"query": {"match_all": {}}})['hits']['hits']
        video_urls = links_qs[0]['_source']['video_url']
        return video_urls


### test

In [158]:
obj = YoutubePlExtracter()

C:\Users\Kaustav\AppData\Local\Temp\ipykernel_1616\3643081683.py:100: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  firefox_option.headless = head_less
C:\Users\Kaustav\AppData\Local\Temp\ipykernel_1616\3643081683.py:108: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Firefox(options=firefox_option,executable_path=selenium_config["windows_path"]["firefox_64_bit"])


In [159]:
result = obj.getlinks(url="https://www.youtube.com/playlist?list=PL3D7BFF1DDBDAAFE5")

In [160]:
df = pd.DataFrame(result)

In [161]:
df

,video_order_number,video_url,video_title,video_author_name,video_view_count,date_of_publish,video_playlist_title,playlist_url,iframe_embedding_text,video_review
0,1,https://www.youtube.com/watch?v=SUOWNXGRc6g&li...,Android Application Development Tutorial - 1 -...,thenewboston,2M views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
1,2,https://www.youtube.com/watch?v=857zrsYZKGo&li...,Android Application Development Tutorial - 2 -...,thenewboston,852K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
2,3,https://www.youtube.com/watch?v=Da1jlmwuW_w&li...,Android Application Development Tutorial - 3 -...,thenewboston,718K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
3,4,https://www.youtube.com/watch?v=MIKl8PX838E&li...,Android Application Development Tutorial - 4 -...,thenewboston,678K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
4,5,https://www.youtube.com/watch?v=sPFUTJgvVpQ&li...,Android Application Development Tutorial - 5 -...,thenewboston,667K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
...,...,...,...,...,...,...,...,...,...,...
190,191,https://www.youtube.com/watch?v=kzRN2U7H6LA&li...,Android Application Development Tutorial - 196...,thenewboston,41K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
191,192,https://www.youtube.com/watch?v=rrRcGz15LcM&li...,Android Application Development Tutorial - 197...,thenewboston,40K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
192,193,https://www.youtube.com/watch?v=Ll8uYcH6Ex8&li...,Android Application Development Tutorial - 198...,thenewboston,60K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
193,194,https://www.youtube.com/watch?v=1j4prh3NAZE&li...,Android Application Development Tutorial - 199...,thenewboston,91K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."


In [162]:
obj_es = ElasticSearch()

In [163]:
result

[{'video_order_number': '1',
  'video_url': 'https://www.youtube.com/watch?v=SUOWNXGRc6g&list=PL3D7BFF1DDBDAAFE5&index=1&pp=iAQB',
  'video_title': 'Android Application Development Tutorial - 1 - Download and Install the Java JDK',
  'video_author_name': 'thenewboston',
  'video_view_count': '2M views',
  'date_of_publish': '11 years ago',
  'video_playlist_title': 'The New Boston - Android',
  'playlist_url': 'https://www.youtube.com/playlist?list=PL3D7BFF1DDBDAAFE5',
  'iframe_embedding_text': '',
  'video_review': [{'comment': '', 'review_date': '', 'review_rating': ''}]},
 {'video_order_number': '2',
  'video_url': 'https://www.youtube.com/watch?v=857zrsYZKGo&list=PL3D7BFF1DDBDAAFE5&index=2&pp=iAQB',
  'video_title': 'Android Application Development Tutorial - 2 - Installing Eclipse and Setting up the ADT',
  'video_author_name': 'thenewboston',
  'video_view_count': '852K views',
  'date_of_publish': '11 years ago',
  'video_playlist_title': 'The New Boston - Android',
  'playlist

In [164]:
obj_es.write_to_es(data=result)

C:\Users\Kaustav\AppData\Local\Temp\ipykernel_1616\1642079041.py:59: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  ES_INSTANCE.indices.create(index=ES_INDEX_NAME, ignore=400, body=es_config)
C:\Users\Kaustav\AppData\Local\Temp\ipykernel_1616\1642079041.py:59: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  ES_INSTANCE.indices.create(index=ES_INDEX_NAME, ignore=400, body=es_config)


Mapping created successfully
Sequential insertion operation started
Uploading data to %s index in elastic search db... yt_pl_v1
Records inserted:   195
Sequential insertion operation completed.... Time taken:  0:00:26.095772


## logic creation segment

In [100]:
url = "https://www.youtube.com/playlist?list=PL3D7BFF1DDBDAAFE5"

In [101]:
obj.driver.get(url)

In [131]:
soup = bs(obj.driver.page_source, features='html.parser')

In [132]:
soup = remove_tags(soup=soup)

In [ ]:
soup

In [133]:
playlist_title = soup.find('yt-formatted-string',{'id':'text'}).text

In [134]:
result = []

for id,x in enumerate(soup.find_all('ytd-playlist-video-renderer',{'class':'ytd-playlist-video-list-renderer'})):
    temp = {}

    temp['_id'] = f"YP_{id}"

    index = x.find('div',{'id':'index-container'})
    order_id = index.getText().strip("\n")
    temp['video_order_number'] = order_id

    link = x.find('a',{'id':'video-title'})
    try:
        href = link.get('href')
        if href.startswith('/watch?'):
            # print(href)
            temp['video_url'] = "https://www.youtube.com" + href
    except:
        temp['video_url'] = ""
    
    try:
        title = link.get('title')
        temp['video_title'] = title
    except:
        temp['video_title'] = ""
    
    channel_obj = x.find('ytd-channel-name').find('a')
    channel_name = channel_obj.getText()
    # print(channel_name)
    temp['video_author_name'] = channel_name

    sep_obj = x.find('yt-formatted-string',{'id': 'video-info'}).find_all('span')
    # print(sep_obj[0].getText())


    temp['video_view_count'] = sep_obj[0].getText()
    temp['date_of_publish'] = sep_obj[2].getText()
    temp['video_playlist_title'] = playlist_title
    temp['playlist_url'] = url
    temp['iframe_embedding_text'] = ""
    temp['video_review'] = [{"comment":"", "review_date":"", "review_rating":""}]
    result.append(temp)

In [135]:
import pandas as pd
df = pd.DataFrame(result)

In [136]:
df

,_id,video_order_number,video_url,video_title,video_author_name,video_view_count,date_of_publish,video_playlist_title,playlist_url,iframe_embedding_text,video_review
0,YP_0,1,https://www.youtube.com/watch?v=SUOWNXGRc6g&li...,Android Application Development Tutorial - 1 -...,thenewboston,2M views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
1,YP_1,2,https://www.youtube.com/watch?v=857zrsYZKGo&li...,Android Application Development Tutorial - 2 -...,thenewboston,852K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
2,YP_2,3,https://www.youtube.com/watch?v=Da1jlmwuW_w&li...,Android Application Development Tutorial - 3 -...,thenewboston,718K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
3,YP_3,4,https://www.youtube.com/watch?v=MIKl8PX838E&li...,Android Application Development Tutorial - 4 -...,thenewboston,678K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
4,YP_4,5,https://www.youtube.com/watch?v=sPFUTJgvVpQ&li...,Android Application Development Tutorial - 5 -...,thenewboston,667K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
...,...,...,...,...,...,...,...,...,...,...,...
190,YP_190,191,https://www.youtube.com/watch?v=kzRN2U7H6LA&li...,Android Application Development Tutorial - 196...,thenewboston,41K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
191,YP_191,192,https://www.youtube.com/watch?v=rrRcGz15LcM&li...,Android Application Development Tutorial - 197...,thenewboston,40K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
192,YP_192,193,https://www.youtube.com/watch?v=Ll8uYcH6Ex8&li...,Android Application Development Tutorial - 198...,thenewboston,60K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
193,YP_193,194,https://www.youtube.com/watch?v=1j4prh3NAZE&li...,Android Application Development Tutorial - 199...,thenewboston,91K views,11 years ago,The New Boston - Android,https://www.youtube.com/playlist?list=PL3D7BFF...,,"[{'comment': '', 'review_date': '', 'review_ra..."
